In [80]:
from dotenv import load_dotenv
from labelbox import Client, Project, Dataset
from typing import Dict
import uuid

In [81]:
load_dotenv('labelbox2.env')
client = Client()

#### Get RebootRX Project Details

In [4]:
projects = client.get_projects(where=(Project.name == "RebootRX"))
project = next(iter(projects))
print(project.name, project.uid)

RebootRX ckjrr0mm6ziit0726vsbhmmc9


#### Get RebootRX Review Project Details

In [82]:
projects = client.get_projects(where=(Project.name == "RebootRX_review"))
new_project = next(iter(projects))
print(new_project.name, new_project.uid)

RebootRX_review ckksi0uqqqx120789jibi5rn7


#### Create RCT Dataset

In [52]:
dataset = client.create_dataset(name="rct_data_set", projects=new_project)

#### Get Dataset Details

In [83]:
for _datasets in new_project.datasets():
    print(_datasets.uid, _datasets.name)

ckksjeoo207p50y8re7iwfabs rct_data_set


In [62]:
dataset = client.get_dataset("ckksjeoo207p50y8re7iwfabs")

#### Check All Available Data

In [43]:
data = next(iter(project.datasets()))
rows = list(data.data_rows())
print(len(rows))

584


In [100]:
selected_data = ['ckjymew2p0pq30ree5avlfckl', 'ckjymew2e0osv0reecbml1vae', 'ckjymew2y0qj70reeh6ixbxu5', 'ckjymew2k0pdz0reeh5nff7cj', 'ckjymew2g0p030ree9f4udau8', 
                 'ckjymew2p0ppj0reegi6fbzbw', 'ckjymew2q0ptf0ree5vcob5rd', 'ckjymew2f0ouv0ree142dakrv', 'ckkga2xiy00103t666o8tz0t1', 'ckjymew2h0p570reebmxw9aak',
                 'ckjymew2r0pyj0ree65jh3c66', 'ckjymew2q0puf0reebfbj9nkr', 'ckjymew2e0ot70ree8myy2r3e', 'ckjymew2i0p6v0reecdz68720', 'ckjymew2f0ov30reed10faygo',
                 'ckjymew2u0qan0ree7ri9bgkk', 'ckjymew2h0p4r0ree6h9a3tk7', 'ckjymew2h0p530ree9fx22kyp', 'ckjymew2y0qi70ree2w9n2bxk', 'ckjymew2k0pdj0ree466td04w',
                 'ckjymew2l0phb0ree5opwgsl6', 'ckjymew2s0q3j0reeakj00ypy', 'ckjymew2i0p7j0reeaglnamrv', 'ckjymew2j0pa30ree9zv9fwen', 'ckjymew2j0pcn0ree01c8brex',
                 'ckjymew2q0pvz0ree4a9rdbdl', 'ckjymew2e0orv0ree5pthbnib', 'ckjymew2t0q5r0ree4yrq3dpg', 'ckjymew2t0q5n0ree80it9bch', 'ckjymew2i0p7f0ree8n3z944f'
                ]

current_row_data = []
counter = 0
for _rows in rows:
    if _rows.uid in selected_data:
        counter +=1
        print(counter)
        print(_rows.uid)
        current_row_data.append(_rows.row_data)
        print(_rows.row_data)
        #data_row = dataset.create_data_row(row_data=_rows.row_data)

1
ckjymew2e0orv0ree5pthbnib
PMID: 24002499  TITLE. Effect of antiviral treatment with nucleotide/nucleoside analogs on postoperative prognosis of hepatitis B virus-related hepatocellular carcinoma: a two-stage longitudinal clinical study. TITLE END DRUG NAME: Adefovir Dipivoxil PURPOSE: Postoperative prognosis of hepatitis B virus (HBV) -related hepatocellular carcinoma (HCC) is poor. The effect of nucleotide/nucleoside analog (NA) treatment on the prognosis has not been fully clarified. PATIENTS AND METHODS: We carried out a two-stage longitudinal study that included a randomized clinical trial (RCT) to evaluate the effect of NA treatment on postoperative prognosis of HBV-HCC. Seven hundred eighty patients (163 in the RCT) were enrolled onto this study following radical hepatectomy. Lamivudine, adefovir dipivoxil, or entecavir were postoperatively administered to antiviral groups. Surgical specimens were examined immunohistochemically for carboxylic acid-terminal truncated HBV X prote

In [73]:
# data = next(iter(project.datasets()))
# rows = list(data.data_rows())
# print(len(rows))
# for _rows in rows:
#     print(_rows.uid)

def turn_on_model_assisted_labeling(client: Client, project_id: str) -> None:
    """
    Turns model assisted labeling on for the given project

    Args:
        client (Client): The client that is connected via API key
        project_id (str): The id of the project
    Returns:
        None

    """
    client.execute("""
         mutation TurnPredictionsOn($proj_id: ID!){
             project(
                 where: {id: $proj_id}
             ){
                 showPredictionsToLabelers(show:true){
                     id
                     showingPredictionsToLabelers
                 }
             }
         }
     """, {"proj_id": project_id})

In [ ]:
turn_on_model_assisted_labeling(client, new_project.uid)

#### Check Labels

In [109]:
import json

data = next(iter(project.datasets()))
rows = list(data.data_rows())
counter = 0
for _rows in rows:
    if _rows.row_data in current_row_data:
        counter +=1
        print('Row UID', _rows.uid)
        print('----------------')
        print('----------------')
        for _label in _rows.labels():
            print('Label ID', _label.uid)
            current_label = json.loads(_label.label)
            if 'objects' in current_label:
                for details in current_label['objects']:
                    print('Title:', details['title'])
                    print('Schema Id:', details['schemaId'])
                    print('Location Start:', details['data']['location']['start'])
                    print('Location End:', details['data']['location']['end'])
                    print('----------------')
print(counter)

Row UID ckjymew2e0orv0ree5pthbnib
----------------
----------------
Label ID ckkijestx00293b68uyxbeva3
Title: PFS_HR
Schema Id: ckjrsr0fc0dos0y83gxf13yh5
Location Start: 1347
Location End: 1350
----------------
Title: OS_HR
Schema Id: ckjrsr0fc0dor0y831nux4l73
Location Start: 1379
Location End: 1382
----------------
Title: group1
Schema Id: ckjrshew80b7i0y7g9zzydsj4
Location Start: 607
Location End: 618
----------------
Title: group2
Schema Id: ckjrshew80b7k0y7gaubc7la9
Location Start: 1637
Location End: 1648
----------------
Title: response_p
Schema Id: ckjrsr0fc0dom0y837ncoet45
Location Start: 1655
Location End: 1658
----------------
Title: response_p
Schema Id: ckjrsr0fc0dom0y837ncoet45
Location Start: 1875
Location End: 1878
----------------
Title: total_sample_size
Schema Id: ckjrsj79q0d550y83cx113wp1
Location Start: 659
Location End: 687
----------------
Title: PFS_p
Schema Id: ckjrsr0fd0dou0y83bajt6aoy
Location Start: 1751
Location End: 1754
----------------
Row UID ckjymew2e0os

Label ID ckkisda7b000w3g65w7spywsl
Title: group1
Schema Id: ckjrshew80b7i0y7g9zzydsj4
Location Start: 719
Location End: 729
----------------
Title: total_sample_size
Schema Id: ckjrsj79q0d550y83cx113wp1
Location Start: 1121
Location End: 1125
----------------
Title: g1_n
Schema Id: ckjrsj79r0d560y83fv2875ab
Location Start: 1137
Location End: 1139
----------------
Title: OS_HR
Schema Id: ckjrsr0fc0dor0y831nux4l73
Location Start: 1388
Location End: 1391
----------------
Title: group2
Schema Id: ckjrshew80b7k0y7gaubc7la9
Location Start: 764
Location End: 786
----------------
Row UID ckjymew2h0p530ree9fx22kyp
----------------
----------------
Label ID ckkic62hn001q2a6fcxx9ahnh
Title: PFS_p
Schema Id: ckjrsr0fd0dou0y83bajt6aoy
Location Start: 1196
Location End: 1199
----------------
Title: survival_p
Schema Id: ckjrsr0f60dof0y83h5nmgyjh
Location Start: 1219
Location End: 1222
----------------
Title: survival_p
Schema Id: ckjrsr0f60dof0y83h5nmgyjh
Location Start: 746
Location End: 750
------

Label ID ckkfibmzu000k2a6fefbkfbcg
Title: total_sample_size
Schema Id: ckjrsj79q0d550y83cx113wp1
Location Start: 919
Location End: 921
----------------
Title: g1_n
Schema Id: ckjrsj79r0d560y83fv2875ab
Location Start: 943
Location End: 944
----------------
Title: g2_n
Schema Id: ckjrsj79r0d570y83aw4xg9t7
Location Start: 974
Location End: 975
----------------
Title: survival_p
Schema Id: ckjrsr0f60dof0y83h5nmgyjh
Location Start: 1386
Location End: 1389
----------------
Title: PFS_p
Schema Id: ckjrsr0fd0dou0y83bajt6aoy
Location Start: 1349
Location End: 1352
----------------
Title: group2
Schema Id: ckjrshew80b7k0y7gaubc7la9
Location Start: 870
Location End: 892
----------------
Title: group2
Schema Id: ckjrshew80b7k0y7gaubc7la9
Location Start: 908
Location End: 912
----------------
Title: group1
Schema Id: ckjrshew80b7i0y7g9zzydsj4
Location Start: 745
Location End: 843
----------------
Title: group1
Schema Id: ckjrshew80b7i0y7g9zzydsj4
Location Start: 855
Location End: 857
--------------

Label ID ckkgh2byu00034568mxgdjb6s
Title: total_sample_size
Schema Id: ckjrsj79q0d550y83cx113wp1
Location Start: 1015
Location End: 1025
----------------
Title: group1
Schema Id: ckjrshew80b7i0y7g9zzydsj4
Location Start: 248
Location End: 272
----------------
Title: group2
Schema Id: ckjrshew80b7k0y7gaubc7la9
Location Start: 918
Location End: 924
----------------
Row UID ckjymew2q0pvz0ree4a9rdbdl
----------------
----------------
Label ID ckkiibutb00153g60j859tl54
Title: g1_n
Schema Id: ckjrsj79r0d560y83fv2875ab
Location Start: 1105
Location End: 1106
----------------
Title: group1
Schema Id: ckjrshew80b7i0y7g9zzydsj4
Location Start: 1108
Location End: 1122
----------------
Title: g2_n
Schema Id: ckjrsj79r0d570y83aw4xg9t7
Location Start: 1134
Location End: 1135
----------------
Title: g1_response_rate
Schema Id: ckjrsr0fa0dog0y837t3ict2q
Location Start: 1125
Location End: 1128
----------------
Title: g2_response_rate
Schema Id: ckjrsr0fb0doj0y8340kk1n1o
Location Start: 1153
Location En